# Spike sorting

In [ ]:
#da fare:
# soglia silhouette >0.4 (non >=)

In [ ]:
final_neurons=[]
from NewLibraryENGcopia import *
import glob
#final=[]
list_dir=glob.glob("/Users/Gaia_1/Desktop/allh5filestutto/*")
output_path='/Users/Gaia_1/Desktop/tesi/Data after SS'
for file in tqdm(list_dir):
    target=1
    stim=0
    if 'health' in file:
        target=0
    if 'stimulation' in file:
        stim=1
    file_name = file.split("/")[-1]
    print(file_name,':','target',target,'stimulation',stim)
    neurons=spike_sorting(file,output_path)
    #df=poiproc(neurons,target,stim)
    #final.append(df)
    final_neurons.append(neurons)

In [1]:
import glob
list_dir=glob.glob("/Users/Gaia_1/Desktop/allh5filestutto/*")
for file in list_dir:
    print(file)

/Users/Gaia_1/Desktop/allh5filestutto/2018-11-27T10-03-29MIP4 BL .h5
/Users/Gaia_1/Desktop/allh5filestutto/2018-11-27T09-38-09MIP 2 BL .h5
/Users/Gaia_1/Desktop/allh5filestutto/2019-01-23T16-22-47Pop3 healthy cortical stimulation modifiedname.h5
/Users/Gaia_1/Desktop/allh5filestutto/2018-11-27T11-24-28MiP3 stimulation.h5
/Users/Gaia_1/Desktop/allh5filestutto/2019-01-23T15-36-26MEA3 healthy cortical stimulation modifiedname.h5
/Users/Gaia_1/Desktop/allh5filestutto/zzdone
/Users/Gaia_1/Desktop/allh5filestutto/2019-01-23T11-41-41Pop1 healthy cortical .h5


# Point Process

In [ ]:
from NewLibraryENGcopia import *
import glob
final=[]
output_path='/Users/Gaia_1/Desktop/tesi/Data after SS'
list_dir=glob.glob(output_path+"/*")
for file in tqdm(list_dir):
    target=1
    stim=0
    if 'health' in file:
        target=0
    if 'stimulation' in file:
        stim=1
    list_neurons = np.genfromtxt(file, delimiter=',')
    df,counter=poiproc(list_neurons,target,stim)
    final.append(df)

In [ ]:
from NewLibraryENGcopia import *

#POPULATION 1 (HEALTHY):
name_data1 = '2019-01-23T11-19-05PoP1 healthy cortical .h5'
complete_string_BL='/Users/Gaia_1/Desktop/allh5files/healthy/healthy_baseline/'+name_data1
name_data2 = '2019-01-23T16-06-32Pop1 healthy cortical .h5'
complete_string_st='/Users/Gaia_1/Desktop/allh5files/healthy/healthy_stimulation/'+name_data2
name_data3 = '2019-01-24T16-00-33Pop1 24hour after.h5'
complete_string_post='/Users/Gaia_1/Desktop/allh5files/healthy/healthy 24hrs later/'+name_data3

# POPULATION 1 (LKRR2):
name_data4='2018-11-27T10-17-41POP 1 BL .h5'
complete_string_BL_LKRR2='/Users/Gaia_1/Desktop/allh5files/LRRK2/lkkr2 baseline/'+name_data4
name_data5='2018-11-27T11-50-12POP 1 control.h5'
complete_string_st_LKRR2='/Users/Gaia_1/Desktop/allh5files/LRRK2/lkkr2 stimulation/'+name_data5
name_data6='2018-11-28T12-22-45Pop1 control.h5'
complete_string_post_LKRR2='/Users/Gaia_1/Desktop/allh5files/LRRK2/lkkr2 24hrs post/'+name_data6

In [ ]:
from NewLibraryENGcopia import *
import glob
final=[]
target=1
stim=1
list_dir=glob.glob("/Users/Gaia_1/Desktop/allh5files/*")
for targets in list_dir:
    #healthy and LKRR2
    if 'healthy' in targets:
        target=0
        print('healthy')
    folder=glob.glob(targets+'/*')
    for phase in folder:
        #BL, stim, 24hrs post
        if 'baseline' in phase:
            stim=0
            print('baseline')
        final_folders=glob.glob(phase+'/*.h5')
        for file in tqdm(final_folders):
            #all files
            file_name = file.split("/")[-1]
            print(file_name,':')
            neurons=spike_sorting(file_name,file,0,'kmeans',4,2)
            df=poiproc(neurons,target,stim)
            final.append(df)

In [ ]:
bins = np.arange(0, .5, 1e-3) 
plt.figure (figsize=(14,10))
hist = np.histogram(ppc_trace['posterior_predictive']['like'].values,bins=bins)
#plt.axis([-0.01,0.13,0,160])
a= plt.hist(ISI_healthy,bins)
plt.plot(hist[1][:-1],hist[0]/1000,linewidth=3)
plt.show()


In [ ]:
lista_samples=[]
for i in list(ppc_trace['posterior_predictive']['like'].values):
    lista_samples.extend(i)        

In [ ]:
counts, _ = np.histogram(ISI_healthy, bins) 
prob_emp = counts / np.sum(counts)
counts, _ = np.histogram(lista_samples, bins) 
prob_model = counts / np.sum(counts)

Femp = np.cumsum(prob_emp)           
Fmodel = np.cumsum(prob_model)          

plt.figure()
plt.plot(bins[:-1], Femp)                
plt.plot(bins[:-1], Fmodel, 'r')       
#plt.xlim([0, 0.2])                  
#plt.xlabel('Time [s]')
#plt.ylabel('CDF')
plt.legend(['Empirical','Model'])
plt.show()

In [ ]:
plt.figure()
Nlow = len(ISI_healthy)  
# Plot the confidence bounds
plt.plot([0, 1], [x + 1.36 / np.sqrt(Nlow) for x in [0, 1]], 'r:')
plt.plot([0, 1], [x - 1.36 / np.sqrt(Nlow) for x in [0, 1]], 'r:')
plt.plot(Femp, Fmodel)
#plt.axis([0, 1, 0, 1])         
#plt.xlabel('Model CDF')
#plt.ylabel('Empirical CDF')
plt.show()


In [ ]:
dataframe = pd.DataFrame()
list_neurons = neurons
counter=0
target=0
print('Original number of neurons: ',len(list_neurons))
for neuron in list_neurons:
    neuron=neuron[neuron>0*10000]
    neuron=neuron[neuron<200*10000]
    print('  Neuron with ',neuron.shape[0],'spikes')
    if neuron.shape[0]>100:

        counter+=1
    else:
        print('    Excluded neuron with n spikes = ',neuron.shape[0])
        continue

    ISI_healthy = np.diff(neuron)/10000


    map_estimate,ppc_trace = this_Bayesian_mixture_model(ISI_healthy)
    #trace(ppc_trace)
    df = pd.DataFrame.from_dict(map_estimate,orient='index')
    dataframe = pd.concat([dataframe,df],axis = 1)
print('Final number of neurons: ',counter)
print('Target = ',target)

In [ ]:
def this_Bayesian_mixture_model(ISI_data):
    with pm.Model() as model:
        ##### WALD DISTRIBUTION (INVERSE GAUSSIAN)
        mu1 = pm.Uniform('mu1',lower=0.01,upper=0.1)
        lam1 = pm.Uniform('lam1',lower=0.001,upper=0.04)
        obs1 = pm.Wald.dist(mu=mu1,lam=lam1)


        mu2 = pm.Uniform('mu2',lower=0,upper=0.2)
        sigma2 = pm.Uniform('sigma2',lower=0.0001,upper=0.5)
        obs2 = pm.TruncatedNormal.dist(mu=mu2, sigma=sigma2, lower=0.0)

        mu3 = pm.Uniform('mu3',lower=0.1,upper=0.6)
        sigma3 = pm.Uniform('sigma3',lower=0.0001,upper=0.5)
        obs3 = pm.TruncatedNormal.dist(mu=mu3, sigma=sigma3, lower=0.0)


        w = pm.Dirichlet('w', a=np.array([1., .4, .4]))
        #w = pm.Dirichlet('w', a=np.array([1., .4]))

        like = pm.Mixture('like', w=w, comp_dists = [obs1, obs2, obs3], observed=ISI_data)
        #like = pm.Mixture('like', w=w, comp_dists = [obs1, obs2], observed=ISI_data)

        step = pm.NUTS(target_accept=0.9)
        trace = pm.sample(step=step,draws=1000,chains=4,tune=1000,cores=4)
        
        ppc_trace = pm.sample_posterior_predictive(trace,model=model)
        if ppc_trace==0:
            print('ppc_trace not succesful')
        '''
        bins = np.arange(0, .5, 1e-3) 
        plt.figure (figsize=(14,10))
        hist = np.histogram(ppc_trace['posterior_predictive']['like'].values,bins=bins)
        #plt.axis([-0.01,0.13,0,160])
        a= plt.hist(ISI_healthy,bins)
        plt.plot(hist[1][:-1],hist[0]/1000,linewidth=3)
        plt.show()

        lista_samples=[]
        for i in list(ppc_trace['posterior_predictive']['like'].values):
            lista_samples.extend(i)        
        
        counts, _ = np.histogram(ISI_healthy, bins) 
        prob_emp = counts / np.sum(counts)
        counts, _ = np.histogram(lista_samples, bins) 
        prob_model = counts / np.sum(counts)

        Femp = np.cumsum(prob_emp)           
        Fmodel = np.cumsum(prob_model)          
        
        plt.figure()
        plt.plot(bins[:-1], Femp)                
        plt.plot(bins[:-1], Fmodel, 'r')       
        #plt.xlim([0, 0.2])                  
        plt.xlabel('Time [s]')
        plt.ylabel('CDF')
        plt.legend(['Empirical','Model'])
        plt.show()
        
        plt.figure()
        Nlow = len(ISI_healthy)  
        # Plot the confidence bounds
        plt.plot([0, 1], [x + 1.36 / np.sqrt(Nlow) for x in [0, 1]], 'r:')
        plt.plot([0, 1], [x - 1.36 / np.sqrt(Nlow) for x in [0, 1]], 'r:')
        plt.plot(Femp, Fmodel)
        #plt.axis([0, 1, 0, 1])         
        plt.xlabel('Model CDF')
        plt.ylabel('Empirical CDF')
        plt.show()
        '''
        
    map_estimate = pm.find_MAP(model=model)
    
    del map_estimate['w_simplex__']
    del map_estimate['mu1_interval__']
    del map_estimate['lam1_interval__']
    del map_estimate['mu2_interval__']
    del map_estimate['sigma2_interval__']
    del map_estimate['mu3_interval__']
    del map_estimate['sigma3_interval__']
    
    map_estimate['w1'] = map_estimate['w'][0]
    map_estimate['w2'] = map_estimate['w'][1]
    map_estimate['w3'] = map_estimate['w'][2]

    del map_estimate['w']


    return map_estimate, ppc_trace